In [1]:
import itertools
import numpy as np
import pandas as pd
import sys
import os
from datetime import timedelta

print(os.getcwd())

/Users/alexander.prams/hodschallenge


In [2]:
in_file = "challenge/data/device_activations_small.csv"

In [3]:
previous_readings = pd.read_csv(in_file)
print(type(previous_readings['time'][0]))
previous_readings['time'] = pd.to_datetime(previous_readings['time'])
device_list = sorted(previous_readings['device'].unique())
n_devices = len(device_list)
print(type(previous_readings['time'][0]))
print(previous_readings.loc[0, :]['time'])
print(previous_readings.loc[1, :]['time'])
print(previous_readings.loc[2, :]['time'])

<class 'str'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2016-07-01 04:23:32
2016-07-01 06:52:57
2016-07-01 06:53:00


In [4]:
print("Number of data points: ", len(previous_readings))

Number of data points:  589


In [5]:
def date_time_same_hour(dt1, dt2):
    same_until_hour = dt1.year == dt2.year and dt1.month == dt2.month and dt1.day == dt2.day and dt1.hour == dt2.hour
    if same_until_hour:
        return True
    else:
        return False


In [6]:
# IDEA: ADD HOUR & DAY AS FEATURE?
# 541 hours with positive label
# Weekday as 7 binary inputs?
def preallocate_features(previous_readings):
    n_devices = previous_readings['device'].nunique()
    print('n_devices = ', n_devices)
    tmp_time_stamp = first_time_stamp = pd.to_datetime(previous_readings['time'][0])
    
    unique_hour_date_times = [first_time_stamp.replace(minute=0, second=0)]
    
    hour_interval_start_end = pd.date_range(first_time_stamp.replace(minute=0, second=0), 
                                            previous_readings['time'][len(previous_readings) - 1].replace(minute=0, second=0), 
                                           freq='H')
    
    features = pd.DataFrame(0, index=np.arange(len(hour_interval_start_end)), 
                            columns=['time', 'weekday', 'hour'] + device_list)
    features['time'] = hour_interval_start_end
    return features
#print("Unique hours: ", len(preallocate_features(previous_readings)))
print(preallocate_features(previous_readings))
    

n_devices =  6
                   time  weekday  hour  device_2  device_3  device_4  \
0   2016-07-01 04:00:00        0     0         0         0         0   
1   2016-07-01 05:00:00        0     0         0         0         0   
2   2016-07-01 06:00:00        0     0         0         0         0   
3   2016-07-01 07:00:00        0     0         0         0         0   
4   2016-07-01 08:00:00        0     0         0         0         0   
5   2016-07-01 09:00:00        0     0         0         0         0   
6   2016-07-01 10:00:00        0     0         0         0         0   
7   2016-07-01 11:00:00        0     0         0         0         0   
8   2016-07-01 12:00:00        0     0         0         0         0   
9   2016-07-01 13:00:00        0     0         0         0         0   
10  2016-07-01 14:00:00        0     0         0         0         0   
11  2016-07-01 15:00:00        0     0         0         0         0   
12  2016-07-01 16:00:00        0     0         0 

In [7]:
def preprocess_features_and_labels(previous_readings):
    """
    Generate features so that we have an array with dimensions [T, n_devices, 1]
    T: time
    n_devices: number of devices
    Last dimension: number of activations per hour
    """
    
    # We preallocate to avoid many appends (append copies according to pandas docs, might become an issue/slow for large data)
    features = preallocate_features(previous_readings)
    labels = pd.DataFrame(0, index=np.arange(len(features) - 1), 
                            columns=device_list)
    for index, row in previous_readings.iterrows():
        if index == 0:
            continue
            
        dt = row['time'].replace(minute=0, second=0)
        feature_idx = features.index[features['time'] == dt]
        # Increment device's counter at time
        features.loc[feature_idx, row['device']] += 1
        labels.loc[feature_idx - 1, row['device']] = 1
            
        features['time']
    
    #Second loop, can we improve here?
    for index, row in features.iterrows():
        features.loc[index, 'weekday'] = row['time'].weekday()
        features.loc[index, 'hour'] = row['time'].hour
        
        
    features.drop('time', axis=1, inplace=True)
    
    return (features, labels)

features, labels = preprocess_features_and_labels(previous_readings)

n_devices =  6


In [8]:
print("Features")
print(features)

Features
     weekday  hour  device_2  device_3  device_4  device_5  device_6  device_7
0          4     4         0         0         0         0         0         0
1          4     5         0         0         0         0         0         0
2          4     6         3         0         0         0         0         0
3          4     7        12         0         4         0         6         0
4          4     8         6         2         3         3         5         0
5          4     9         4         1         0         0         0         0
6          4    10         4         5         0         0         0         0
7          4    11         5         1         0         0         0         0
8          4    12         8         4         0         0         0         0
9          4    13         7         5         0         0         0         0
10         4    14         4         3         0         0         0         0
11         4    15         9         4     

In [9]:
print("Labels")
print(labels)

Labels
     device_2  device_3  device_4  device_5  device_6  device_7
0           0         0         0         0         0         0
1           1         0         0         0         0         0
2           1         0         1         0         1         0
3           1         1         1         1         1         0
4           1         1         0         0         0         0
5           1         1         0         0         0         0
6           1         1         0         0         0         0
7           1         1         0         0         0         0
8           1         1         0         0         0         0
9           1         1         0         0         0         0
10          1         1         0         0         0         0
11          1         0         0         0         0         0
12          0         0         0         0         0         0
13          0         0         0         0         0         0
14          0         0         0

In [144]:
def create_timeseries_batches(features, labels, length=31):
    np_features = features.to_numpy()[:-1]
    np_labels = labels.to_numpy()
    
    print(np_features.shape)
    print(np_labels.shape)
    
    np_features = np_features[:length*(len(np_features)//length)]
    np_labels = np_labels[:length*(len(np_labels)//length)]
    
    print(np_features.shape)
    print(np_labels.shape)
    
    feature_batch = np.reshape(np_features, [-1, length, np_features.shape[1]])
    label_batch = np.reshape(np_labels, [-1, length, np_labels.shape[1]])
    
    print(feature_batch.shape)
    print(label_batch.shape)
    
    return feature_batch, label_batch

feature_batch, label_batch = create_timeseries_batches(features, labels)
train_feature_batch = feature_batch[:-1]
test_feature_batch = feature_batch[-1]
train_label_batch = label_batch[:-1]
test_label_batch = label_batch[-1]

(125, 8)
(125, 6)
(124, 8)
(124, 6)
(4, 31, 8)
(4, 31, 6)


In [91]:
def calc_ratio_positive_outputs_per_device(labels):
    np_labels = labels.to_numpy()[:-1]
    print(np_labels.shape)
    ratio_per_device = np.sum(np_labels, axis=0) / np_labels.shape[0]
    print("Ratio per device: ", ratio_per_device)
    return np.array(ratio_per_device)
ratio_positive_outputs_per_device = calc_ratio_positive_outputs_per_device(labels)

(124, 6)
Ratio per device:  [0.23387097 0.24193548 0.16935484 0.13709677 0.2016129  0.05645161]


In [92]:
import matplotlib.pyplot as plt
import math
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras import objectives
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [116]:
USE_WEIGHTED_LOSS = True

In [117]:
# Create our own weighted loss to combat label imbalance
def weighted_loss(y_true, y_pred):
    y_pred = K.clip(y_pred, 1e-10, 1.0-1e-10)
    out = -(y_true * K.log(y_pred) / ratio_positive_outputs_per_device + (1.0 - y_true) * K.log(1.0 - y_pred))
    return K.mean(out, axis=-1)

In [139]:
BATCH_SIZE=1

model = Sequential()
model.add(LSTM(32, batch_input_shape=(BATCH_SIZE, None, n_devices + 2), return_sequences=True, stateful=True))
model.add(Dense(n_devices, activation='sigmoid'))
model.compile(loss=weighted_loss if USE_WEIGHTED_LOSS else 'binary_crossentropy', optimizer='adam')
model.fit(train_feature_batch, train_label_batch, epochs=1000, batch_size=BATCH_SIZE, verbose=2, shuffle=False)

Epoch 1/1000
 - 3s - loss: 1.2945
Epoch 2/1000
 - 0s - loss: 1.2582
Epoch 3/1000
 - 0s - loss: 1.2182
Epoch 4/1000
 - 0s - loss: 1.1806
Epoch 5/1000
 - 0s - loss: 1.1451
Epoch 6/1000
 - 0s - loss: 1.1121
Epoch 7/1000
 - 0s - loss: 1.0819
Epoch 8/1000
 - 0s - loss: 1.0540
Epoch 9/1000
 - 0s - loss: 1.0271
Epoch 10/1000
 - 0s - loss: 1.0004
Epoch 11/1000
 - 0s - loss: 0.9734
Epoch 12/1000
 - 0s - loss: 0.9458
Epoch 13/1000
 - 0s - loss: 0.9182
Epoch 14/1000
 - 0s - loss: 0.8906
Epoch 15/1000
 - 0s - loss: 0.8630
Epoch 16/1000
 - 0s - loss: 0.8353
Epoch 17/1000
 - 0s - loss: 0.8080
Epoch 18/1000
 - 0s - loss: 0.7812
Epoch 19/1000
 - 0s - loss: 0.7552
Epoch 20/1000
 - 0s - loss: 0.7304
Epoch 21/1000
 - 0s - loss: 0.7072
Epoch 22/1000
 - 0s - loss: 0.6854
Epoch 23/1000
 - 0s - loss: 0.6641
Epoch 24/1000
 - 0s - loss: 0.6440
Epoch 25/1000
 - 0s - loss: 0.6247
Epoch 26/1000
 - 0s - loss: 0.6058
Epoch 27/1000
 - 0s - loss: 0.5877
Epoch 28/1000
 - 0s - loss: 0.5707
Epoch 29/1000
 - 0s - loss: 0

Epoch 232/1000
 - 0s - loss: 0.1094
Epoch 233/1000
 - 0s - loss: 0.1089
Epoch 234/1000
 - 0s - loss: 0.1083
Epoch 235/1000
 - 0s - loss: 0.1079
Epoch 236/1000
 - 0s - loss: 0.1073
Epoch 237/1000
 - 0s - loss: 0.1069
Epoch 238/1000
 - 0s - loss: 0.1065
Epoch 239/1000
 - 0s - loss: 0.1061
Epoch 240/1000
 - 0s - loss: 0.1057
Epoch 241/1000
 - 0s - loss: 0.1052
Epoch 242/1000
 - 0s - loss: 0.1048
Epoch 243/1000
 - 0s - loss: 0.1045
Epoch 244/1000
 - 0s - loss: 0.1041
Epoch 245/1000
 - 0s - loss: 0.1038
Epoch 246/1000
 - 0s - loss: 0.1035
Epoch 247/1000
 - 0s - loss: 0.1033
Epoch 248/1000
 - 0s - loss: 0.1030
Epoch 249/1000
 - 0s - loss: 0.1026
Epoch 250/1000
 - 0s - loss: 0.1024
Epoch 251/1000
 - 0s - loss: 0.1021
Epoch 252/1000
 - 0s - loss: 0.1019
Epoch 253/1000
 - 0s - loss: 0.1016
Epoch 254/1000
 - 0s - loss: 0.1014
Epoch 255/1000
 - 0s - loss: 0.1012
Epoch 256/1000
 - 0s - loss: 0.1010
Epoch 257/1000
 - 0s - loss: 0.1008
Epoch 258/1000
 - 0s - loss: 0.1006
Epoch 259/1000
 - 0s - loss:

Epoch 460/1000
 - 0s - loss: 0.0825
Epoch 461/1000
 - 0s - loss: 0.0828
Epoch 462/1000
 - 0s - loss: 0.0819
Epoch 463/1000
 - 0s - loss: 0.0851
Epoch 464/1000
 - 0s - loss: 0.0821
Epoch 465/1000
 - 0s - loss: 0.0830
Epoch 466/1000
 - 0s - loss: 0.0829
Epoch 467/1000
 - 0s - loss: 0.0832
Epoch 468/1000
 - 0s - loss: 0.0816
Epoch 469/1000
 - 0s - loss: 0.0817
Epoch 470/1000
 - 0s - loss: 0.0811
Epoch 471/1000
 - 0s - loss: 0.0825
Epoch 472/1000
 - 0s - loss: 0.0818
Epoch 473/1000
 - 0s - loss: 0.0817
Epoch 474/1000
 - 0s - loss: 0.0816
Epoch 475/1000
 - 0s - loss: 0.0814
Epoch 476/1000
 - 0s - loss: 0.0815
Epoch 477/1000
 - 0s - loss: 0.0817
Epoch 478/1000
 - 0s - loss: 0.0837
Epoch 479/1000
 - 0s - loss: 0.0861
Epoch 480/1000
 - 0s - loss: 0.0853
Epoch 481/1000
 - 0s - loss: 0.0846
Epoch 482/1000
 - 0s - loss: 0.0851
Epoch 483/1000
 - 0s - loss: 0.0872
Epoch 484/1000
 - 0s - loss: 0.0862
Epoch 485/1000
 - 0s - loss: 0.0882
Epoch 486/1000
 - 0s - loss: 0.1057
Epoch 487/1000
 - 0s - loss:

Epoch 688/1000
 - 0s - loss: 0.0556
Epoch 689/1000
 - 0s - loss: 0.0554
Epoch 690/1000
 - 0s - loss: 0.0552
Epoch 691/1000
 - 0s - loss: 0.0550
Epoch 692/1000
 - 0s - loss: 0.0549
Epoch 693/1000
 - 0s - loss: 0.0547
Epoch 694/1000
 - 0s - loss: 0.0545
Epoch 695/1000
 - 0s - loss: 0.0542
Epoch 696/1000
 - 0s - loss: 0.0541
Epoch 697/1000
 - 0s - loss: 0.0539
Epoch 698/1000
 - 0s - loss: 0.0537
Epoch 699/1000
 - 0s - loss: 0.0535
Epoch 700/1000
 - 0s - loss: 0.0533
Epoch 701/1000
 - 0s - loss: 0.0531
Epoch 702/1000
 - 0s - loss: 0.0528
Epoch 703/1000
 - 0s - loss: 0.0527
Epoch 704/1000
 - 0s - loss: 0.0525
Epoch 705/1000
 - 0s - loss: 0.0523
Epoch 706/1000
 - 0s - loss: 0.0521
Epoch 707/1000
 - 0s - loss: 0.0518
Epoch 708/1000
 - 0s - loss: 0.0516
Epoch 709/1000
 - 0s - loss: 0.0514
Epoch 710/1000
 - 0s - loss: 0.0512
Epoch 711/1000
 - 0s - loss: 0.0510
Epoch 712/1000
 - 0s - loss: 0.0508
Epoch 713/1000
 - 0s - loss: 0.0506
Epoch 714/1000
 - 0s - loss: 0.0504
Epoch 715/1000
 - 0s - loss:

Epoch 916/1000
 - 0s - loss: 0.0244
Epoch 917/1000
 - 0s - loss: 0.0243
Epoch 918/1000
 - 0s - loss: 0.0242
Epoch 919/1000
 - 0s - loss: 0.0241
Epoch 920/1000
 - 0s - loss: 0.0241
Epoch 921/1000
 - 0s - loss: 0.0240
Epoch 922/1000
 - 0s - loss: 0.0239
Epoch 923/1000
 - 0s - loss: 0.0239
Epoch 924/1000
 - 0s - loss: 0.0238
Epoch 925/1000
 - 0s - loss: 0.0237
Epoch 926/1000
 - 0s - loss: 0.0237
Epoch 927/1000
 - 0s - loss: 0.0235
Epoch 928/1000
 - 0s - loss: 0.0234
Epoch 929/1000
 - 0s - loss: 0.0233
Epoch 930/1000
 - 0s - loss: 0.0232
Epoch 931/1000
 - 0s - loss: 0.0232
Epoch 932/1000
 - 0s - loss: 0.0231
Epoch 933/1000
 - 0s - loss: 0.0230
Epoch 934/1000
 - 0s - loss: 0.0229
Epoch 935/1000
 - 0s - loss: 0.0229
Epoch 936/1000
 - 0s - loss: 0.0228
Epoch 937/1000
 - 0s - loss: 0.0227
Epoch 938/1000
 - 0s - loss: 0.0226
Epoch 939/1000
 - 0s - loss: 0.0226
Epoch 940/1000
 - 0s - loss: 0.0225
Epoch 941/1000
 - 0s - loss: 0.0225
Epoch 942/1000
 - 0s - loss: 0.0224
Epoch 943/1000
 - 0s - loss:

In [146]:
predictions = model.predict(np.expand_dims(test_feature_batch, axis=0))
print(np.round(predictions, 1))

[[[0.9 0.8 0.4 0.  0.9 0.3]
  [0.9 0.5 0.5 0.  0.9 0.1]
  [1.  0.3 0.9 0.1 0.9 0.1]
  [1.  0.2 1.  0.6 0.9 0.3]
  [1.  0.2 1.  0.6 1.  0.4]
  [0.9 0.2 1.  0.8 0.9 0.2]
  [0.7 0.1 1.  1.  0.9 0.2]
  [0.4 0.5 1.  1.  1.  0. ]
  [0.7 0.5 1.  1.  1.  0. ]
  [0.7 0.7 1.  1.  1.  0. ]
  [0.7 0.9 1.  1.  1.  0. ]
  [0.6 1.  1.  1.  1.  0. ]
  [0.9 1.  1.  1.  1.  0. ]
  [1.  1.  1.  1.  1.  0. ]
  [1.  1.  1.  1.  1.  0. ]
  [1.  1.  1.  1.  1.  1. ]
  [0.8 1.  0.3 0.1 1.  0. ]
  [0.5 1.  0.  0.  0.4 0. ]
  [0.1 1.  0.  0.  0.1 0. ]
  [0.  0.9 0.  0.  0.  0. ]
  [0.  0.7 0.  0.  0.  0. ]
  [0.  0.4 0.  0.  0.  0. ]
  [0.  0.1 0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0. ]
  [0.1 0.  0.  0.  0.  0. ]
  [0.4 0.  0.  0.  0.1 0. ]
  [1.  0.2 0.1 0.  0.9 0. ]
  [1.  1.  0.8 0.  1.  0. ]
  [1.  1.  1.  1.  1.  1. ]]]


In [150]:
print("Training accuracy: ", np.sum(np.round(predictions) == test_label_batch) / predictions.size) 
print("Training accuracy per device:: ", np.sum(np.round(predictions) == test_label_batch, axis=1) / predictions.shape[1]) 

print("% of 1 prediction outputs", np.sum(np.round(predictions)) / predictions.size) 
print("% of 1 label outputs", np.sum(np.round(test_label_batch)) / test_label_batch.size)

Training accuracy:  0.8225806451612904
Training accuracy per device::  [[0.5483871  0.80645161 0.87096774 0.90322581 0.87096774 0.93548387]]
% of 1 prediction outputs 0.4731182795698925
% of 1 label outputs 0.3172043010752688


In [135]:
print(labels)

     device_2  device_3  device_4  device_5  device_6  device_7
0           0         0         0         0         0         0
1           1         0         0         0         0         0
2           1         0         1         0         1         0
3           1         1         1         1         1         0
4           1         1         0         0         0         0
5           1         1         0         0         0         0
6           1         1         0         0         0         0
7           1         1         0         0         0         0
8           1         1         0         0         0         0
9           1         1         0         0         0         0
10          1         1         0         0         0         0
11          1         0         0         0         0         0
12          0         0         0         0         0         0
13          0         0         0         0         0         0
14          0         0         0       

In [142]:
model.evaluate(feature_batch, label_batch)

ValueError: In a stateful network, you should only pass inputs with a number of samples that can be divided by the batch size. Found: 4 samples